# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4fc8258310>
├── 'a' --> tensor([[-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270]])
└── 'x' --> <FastTreeValue 0x7f4fc8258af0>
    └── 'c' --> tensor([[ 0.2794, -1.6595, -0.5230, -0.3649],
                        [ 0.8566,  1.1762, -1.4426,  0.8819],
                        [ 1.6697,  0.1821, -0.6233, -1.4758]])

In [4]:
t.a

tensor([[-0.9205,  1.0151, -0.8561],
        [-0.4746, -0.5300,  0.0270]])

In [5]:
%timeit t.a

76 ns ± 0.118 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4fc8258310>
├── 'a' --> tensor([[ 0.1976, -0.5824, -0.6382],
│                   [ 0.1245,  0.1051, -0.2407]])
└── 'x' --> <FastTreeValue 0x7f4fc8258af0>
    └── 'c' --> tensor([[ 0.2794, -1.6595, -0.5230, -0.3649],
                        [ 0.8566,  1.1762, -1.4426,  0.8819],
                        [ 1.6697,  0.1821, -0.6233, -1.4758]])

In [7]:
%timeit t.a = new_value

87 ns ± 0.145 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9205,  1.0151, -0.8561],
               [-0.4746, -0.5300,  0.0270]]),
    x: Batch(
           c: tensor([[ 0.2794, -1.6595, -0.5230, -0.3649],
                      [ 0.8566,  1.1762, -1.4426,  0.8819],
                      [ 1.6697,  0.1821, -0.6233, -1.4758]]),
       ),
)

In [10]:
b.a

tensor([[-0.9205,  1.0151, -0.8561],
        [-0.4746, -0.5300,  0.0270]])

In [11]:
%timeit b.a

69.8 ns ± 2.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5721,  0.3844,  0.0863],
               [ 1.7426, -1.0054,  0.0490]]),
    x: Batch(
           c: tensor([[ 0.2794, -1.6595, -0.5230, -0.3649],
                      [ 0.8566,  1.1762, -1.4426,  0.8819],
                      [ 1.6697,  0.1821, -0.6233, -1.4758]]),
       ),
)

In [13]:
%timeit b.a = new_value

594 ns ± 0.396 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

975 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 62.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

163 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4f1913f670>
├── 'a' --> tensor([[[-0.9205,  1.0151, -0.8561],
│                    [-0.4746, -0.5300,  0.0270]],
│           
│                   [[-0.9205,  1.0151, -0.8561],
│                    [-0.4746, -0.5300,  0.0270]],
│           
│                   [[-0.9205,  1.0151, -0.8561],
│                    [-0.4746, -0.5300,  0.0270]],
│           
│                   [[-0.9205,  1.0151, -0.8561],
│                    [-0.4746, -0.5300,  0.0270]],
│           
│                   [[-0.9205,  1.0151, -0.8561],
│                    [-0.4746, -0.5300,  0.0270]],
│           
│                   [[-0.9205,  1.0151, -0.8561],
│                    [-0.4746, -0.5300,  0.0270]],
│           
│                   [[-0.9205,  1.0151, -0.8561],
│                    [-0.4746, -0.5300,  0.0270]],
│           
│                   [[-0.9205,  1.0151, -0.8561],
│                    [-0.4746, -0.5300,  0.0270]]])
└── 'x' --> <FastTreeValue 0x7f4f1913f940>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.1 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4f124c3d00>
├── 'a' --> tensor([[-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270],
│                   [-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270],
│                   [-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270],
│                   [-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270],
│                   [-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270],
│                   [-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270],
│                   [-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270],
│                   [-0.9205,  1.0151, -0.8561],
│                   [-0.4746, -0.5300,  0.0270]])
└── 'x' --> <FastTreeValue 0x7f4f23f80e80>
    └── 'c' --> tensor([[ 0.2794, -1.6595, -0.5230, -0.3649],
                        [ 0.8566,  1.1762, -1.4426,  0.8819],
                 

In [23]:
%timeit t_cat(trees)

37 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.5 µs ± 607 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2794, -1.6595, -0.5230, -0.3649],
                       [ 0.8566,  1.1762, -1.4426,  0.8819],
                       [ 1.6697,  0.1821, -0.6233, -1.4758]],
              
                      [[ 0.2794, -1.6595, -0.5230, -0.3649],
                       [ 0.8566,  1.1762, -1.4426,  0.8819],
                       [ 1.6697,  0.1821, -0.6233, -1.4758]],
              
                      [[ 0.2794, -1.6595, -0.5230, -0.3649],
                       [ 0.8566,  1.1762, -1.4426,  0.8819],
                       [ 1.6697,  0.1821, -0.6233, -1.4758]],
              
                      [[ 0.2794, -1.6595, -0.5230, -0.3649],
                       [ 0.8566,  1.1762, -1.4426,  0.8819],
                       [ 1.6697,  0.1821, -0.6233, -1.4758]],
              
                      [[ 0.2794, -1.6595, -0.5230, -0.3649],
                       [ 0.8566,  1.1762, -1.4426,  0.8819],
                       [ 1.6697,  0.1821, -0.6233, -1.4758]],

In [26]:
%timeit Batch.stack(batches)

89.2 µs ± 434 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2794, -1.6595, -0.5230, -0.3649],
                      [ 0.8566,  1.1762, -1.4426,  0.8819],
                      [ 1.6697,  0.1821, -0.6233, -1.4758],
                      [ 0.2794, -1.6595, -0.5230, -0.3649],
                      [ 0.8566,  1.1762, -1.4426,  0.8819],
                      [ 1.6697,  0.1821, -0.6233, -1.4758],
                      [ 0.2794, -1.6595, -0.5230, -0.3649],
                      [ 0.8566,  1.1762, -1.4426,  0.8819],
                      [ 1.6697,  0.1821, -0.6233, -1.4758],
                      [ 0.2794, -1.6595, -0.5230, -0.3649],
                      [ 0.8566,  1.1762, -1.4426,  0.8819],
                      [ 1.6697,  0.1821, -0.6233, -1.4758],
                      [ 0.2794, -1.6595, -0.5230, -0.3649],
                      [ 0.8566,  1.1762, -1.4426,  0.8819],
                      [ 1.6697,  0.1821, -0.6233, -1.4758],
                      [ 0.2794, -1.6595, -0.5230, -0.3649],
                   

In [28]:
%timeit Batch.cat(batches)

163 µs ± 449 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

393 µs ± 22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
